# Getting RIVM stats on COVID-19 cases in NL and send it via Telegram 

Scrapping the page:

In [1]:
import pandas as pd
import cfscrape
from lxml import etree

url="https://www.rivm.nl/coronavirus-kaart-van-nederland-per-gemeente"

header = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9',
          'Accept-Encoding': 'gzip, deflate, sdch',
          'Accept-Language' : 'nl-NL,nl;q=0.8,en-US;q=0.6,en;q=0.4',
          'Cache-Control' : 'max-age=0',
          'Connection': 'keep-alive',
          'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.81 Safari/537.36'}

scraper = cfscrape.create_scraper()
scraped_html=scraper.get(url,headers=header).content
html = etree.HTML(scraped_html)

date = html.xpath("//div[@id='mapTitles']/text()")[0].split('tot en met ')[1].split('"')[0]
date = pd.to_datetime(date, format='%d-%m-%Y').strftime('%d-%m-%Y')
time = html.xpath("//p/text()")[0].split()[5]

data = html.xpath("//div[@id='csvData']/text()")

print("Last update from the RIVM page:",date,time)

Last update from the RIVM page: 22-03-2020 14.00


Loading the data in a dataframe:

In [4]:
import pandas as pd
import io
import numpy as np
df = pd.read_csv(io.StringIO('\n'.join(str(data[0]).split('\n')[1:])), sep=';')

Workaround to get cases in 'unknown' municipalities:

In [5]:
import re
aantal_unknown_gemeente = int(re.findall(r'\d+',html.xpath("//div[@class='container container-spacer-sm content nobg clearfix']/p/text()")[3])[0])
df.loc[0,'Gemeente']='Unknown'
df.loc[0,'Aantal']= aantal_unknown_gemeente

In [6]:
df.head()

,Gemnr,Gemeente,Aantal,BevAant,Aantal per 100.000 inwoners
0,-1,Unknown,155,NaN,NaN
1,518,'s-Gravenhage,47,537833.0,8.7
2,796,'s-Hertogenbosch,55,154205.0,35.7
3,1680,Aa en Hunze,1,25386.0,3.9
4,358,Aalsmeer,7,31728.0,22.1


Loading previous day:

In [7]:
previous_day = (pd.to_datetime(date, format='%d-%m-%Y') - np.timedelta64(1, 'D')).strftime('%d-%m-%Y')
df_previous = pd.read_csv("data/covid19-nl-"+str(previous_day)+".csv")
previous_day

'21-03-2020'

Creating a list of requested Gemeentes:

In [8]:
df[df['Gemeente'].str.contains('Tilburg')]

,Gemnr,Gemeente,Aantal,BevAant,Aantal per 100.000 inwoners
286,855,Tilburg,178,217259.0,81.9


In [9]:
gemeentes_requested=['Utrecht',
                    'Enschede',
                    'Haarlemmermeer',
                    'Houten',
                    'Leiden',
                    'Arnhem',
                    'Ridderkerk',
                    'Zuidplas',
                    'Nieuwegein',
                    'Leusden',
                    'Rheden',
                    'Amersfoort',
                    'Woerden',
                    'Epe',
                    'Altena',
                    'Apeldoorn',
                    'Nijmegen',
                    'Zoetermeer',
                    'Breda',
                    'Zwolle',
                    'Montferland',
                     'Doetinchem',
                    'Oude IJsselstreek',
                    'Bergen op Zoom',
                    'Amstelveen',
                    'Meppel',
                    'Zwartewaterland',
                    '\'s-Gravenhage',
                    'Oost Gelre',
                    'Haarlem',
                    'Amsterdam',
                    'Tiel',
                    'Hengelo (O)',
                    '\'s-Hertogenbosch',
                    'Tilburg']
gemeentes_requested.sort()

Composing the message:

In [10]:
message="COVID19 CASES IN THE NETHERLANDS\n"
message+="HOURLY MESSAGES (From 8-21)!\n"
message+="Source: https://www.rivm.nl/coronavirus-kaart-van-nederland-per-gemeente\n"
message+="RIVM last update: "+str(date)+" "+ str(time)+"\n"

message+="\n PREVIOUS >> CURRENT NUMBERS\n"

aantal_total = df['Aantal'].sum()
message+="\n- TOTAL CASES IN NL: " + str(df_previous['Aantal'].sum()) + " >> "+str(aantal_total) +"\n"

for gemeente in gemeentes_requested:
    message += "- Cases in "+gemeente+": "+str(df_previous[df_previous['Gemeente']==gemeente]['Aantal'].values[0])+" >> "+str(df[df['Gemeente']==gemeente]['Aantal'].values[0])+"\n"

message+="\nNote: RIVM stated that \"the actual number of infections with COVID-19 is higher than the number of reports in this update because not everyone suspected of a COVID-19 infection is tested.\"\n"
message+="\nTELEGRAM FAILS TO SHOW ALL 355 GEMEENTES!\nWhich municipality should I add here?\nYour request will appear in the next hour."
print(message)          


COVID19 CASES IN THE NETHERLANDS
HOURLY MESSAGES (From 8-21)!
Source: https://www.rivm.nl/coronavirus-kaart-van-nederland-per-gemeente
RIVM last update: 22-03-2020 14.00

 PREVIOUS >> CURRENT NUMBERS

- TOTAL CASES IN NL: 3631 >> 4204
- Cases in 's-Gravenhage: 44 >> 47
- Cases in 's-Hertogenbosch: 45 >> 55
- Cases in Altena: 22 >> 23
- Cases in Amersfoort: 37 >> 37
- Cases in Amstelveen: 26 >> 27
- Cases in Amsterdam: 161 >> 188
- Cases in Apeldoorn: 16 >> 17
- Cases in Arnhem: 15 >> 16
- Cases in Bergen op Zoom: 9 >> 10
- Cases in Breda: 133 >> 140
- Cases in Doetinchem: 5 >> 5
- Cases in Enschede: 16 >> 18
- Cases in Epe: 6 >> 8
- Cases in Haarlem: 25 >> 27
- Cases in Haarlemmermeer: 12 >> 16
- Cases in Hengelo (O): 13 >> 15
- Cases in Houten: 19 >> 23
- Cases in Leiden: 16 >> 16
- Cases in Leusden: 6 >> 6
- Cases in Meppel: 1 >> 1
- Cases in Montferland: 1 >> 2
- Cases in Nieuwegein: 9 >> 9
- Cases in Nijmegen: 60 >> 68
- Cases in Oost Gelre: 2 >> 2
- Cases in Oude IJsselstreek: 4 >

Function to send a Telegram message:

In [11]:
import requests
def telegram_bot_sendtext(bot_message,token,chatid):
    bot_token = token
    bot_chatID = chatid
    send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + bot_message
    response = requests.get(send_text)
    if response.json()['ok']:
        return "Message Sent!"
    else:
        return "Message failed to be sent!"

Instantiating the function and sending a Telegram message:

In [12]:
telegram_bot_sendtext(message,bot_token,bot_chatID)

'Message Sent!'

Saving the data if it is a new day:

In [13]:
import pathlib
file_name = pathlib.Path('/Users/santannajj/Desktop/crawling_stats_from_rivm_covid19/data/covid19-nl-'+date+'.csv')
if not file_name.exists ():
    df.to_csv(file_name, index = False)
else:
    print("We already have this data!")

We already have this data!


Crontab line:
```
5 08-21 * * * /usr/local/bin/python3 /Users/santannajj/Desktop/crawling_stats_from_rivm_covid19/crawling_stats_from_rivm_covid19.py
```